<div style="text-align: right"> &uarr;   Ensure Kernel is set to  &uarr;  </div><br><div style="text-align: right"> 
conda_amazonei_pytorch_latest_p36  </div>

In [ ]:
!pip install -U sagemaker
#restart your kernel

# FastAI Training using SageMaker Bring your own Container (BYOC)

In this notebook, we will cover how to bring our own container with either a framework or algorithm to train a model on SageMaker. 

We will use fastai in this case and build our container with custom training code integrated into the container. The other option is to use script mode which is easily done by changing the entrypoint.

The outline of this notebook is 

1. Build docker a image for FastAI and serving and training code (provided).

2. Log into ECR, tag and push docker image to ECR 

3. Use the FastAI container image in SageMaker to train our model 

4. Deploy model to endpoint using the container image

5. Test inference using an image in couple of possible ways 

#### Container Image
Let's start with building a container image locally and then push that to ECR (Elastic Container Registry)

In [ ]:
%cd ~/SageMaker/pssummitwkshp/byoc/docker

In [ ]:
!docker build -t fastai .

In [ ]:
!docker images

## Set the ecr details and tags 
Lets set a few params here like ecr name space , tag name etc.

In [ ]:
from sagemaker import get_execution_role
import boto3
ecr_namespace = "sagemaker-training-containers/"
prefix = "script-mode-container-fastai"

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(":")[4]
region = boto3.Session().region_name
tag_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/'+ecr_repository_name+':latest'

In [ ]:
tag_name

Now we tag our image with the tag name we generated above

In [ ]:
!docker tag fastai $tag_name

### ECR Repository and push steps

All of these can be scripted out but they are laid out this way for transparency and step evolution understanding

First we get a token credential to ECR. This will allow us to perform ECR operations

In [ ]:
!$(aws ecr get-login --no-include-email)

Here we create an ECR repository

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository_name

Now that our ECR respoitory has been created, we can now push our docker image to it with the tag name we assigned to it

In [ ]:
!docker push $tag_name

This is how we get the URI of our uploaded docker image in ECR

In [ ]:
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, ecr_repository_name
)
print(container_image_uri)

#### Call your custom container to train the model

In the cell below, replace **your-unique-bucket-name** with the name of bucket you created in the data-prep notebook<br>
**Note:** This cell takes around **20 mins** to run

In [ ]:
%%time
import sagemaker
import json

bucket = "your-unique-bucket-name"


# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}


hyperparameters = json_encode_hyperparameters({"lr":1e-03})

est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    instance_count=1,
    instance_type='ml.m5.12xlarge',
    base_job_name=prefix,
    hyperparameters=hyperparameters,
)

train_config = sagemaker.session.TrainingInput(f's3://{bucket}/train')

est.fit({"train": train_config})

Finally let us print out the trained FastAI model location. You will need this information for the inference step

In [ ]:
print(f'FastAI Model located at \n{est.output_path}{est._current_job_name}/output/model.tar.gz')

### Attach to a training job that has been left to run 

If your kernel becomes disconnected and your training has already started, you can reattach to the training job.<br>
Simply look up the training job name and replace the **your-training-job-name** and then run the cell below. <br>
Once the training job is finished, you can continue the cells after the training cell

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

training_job_name = 'your-training-job-name'

if training_job_name != 'your-training-job-name':
    est = sagemaker.estimator.Estimator.attach(training_job_name=training_job_name, sagemaker_session=sess)